# Cloud Workshop Azure Databricks
## 05. Modèle de régression

## Creating a Regression Model

In this exercise, you will implement a regression model that uses features of a flight to predict how late or early it will arrive.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating how many minutes late or early the flight arrived.

You will load this data into a DataFrame and display it.

In [5]:
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)


### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [7]:
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

train_rows = train.count()
test_rows = test.count()

print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 445988 Testing Rows: 191095

In [8]:
display(train)

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
1,1,9E,10397,12264,94,113,1
1,1,9E,10397,13851,2,2,0
1,1,9E,10721,11193,-5,-2,0
1,1,9E,11066,10721,13,7,0
1,1,9E,11066,13487,-1,-12,0
1,1,9E,11193,10529,31,27,1
1,1,9E,11193,10821,62,56,1
1,1,9E,11193,11278,-3,62,1
1,1,9E,11193,11298,55,55,1
1,1,9E,11193,13198,4,21,0


In [9]:
display(test)

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
1,1,9E,10529,14492,20,41,1
1,1,9E,10721,12478,25,30,1
1,1,9E,10721,12478,37,30,1
1,1,9E,11066,12478,96,82,1
1,1,9E,11193,10529,16,86,1
1,1,9E,11193,13342,-3,-10,0
1,1,9E,11193,13930,60,54,1
1,1,9E,12478,10821,29,0,0
1,1,9E,12478,11433,-1,-28,0
1,1,9E,13244,10423,-6,-8,0


### Prepare the Training Data
To train the regression model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **StringIndexer** class to generate a numeric category for each discrete **Carrier** string value, and then use the **VectorAssembler** class to transform the numeric features that would be available for a flight that hasn't yet arrived into a vector, and then rename the **ArrDelay** column to **label** as this is what we're going to try to predict.

*Note: This is a deliberately simple example. In reality you'd likely perform mulitple data preparation steps, and later in this course we'll examine how to encapsulate these steps in to a pipeline. For now, we'll just use the numeric features as they are to dewfine the traaining dataset.*

In [11]:
# Carrier is a string, and we need our features to be numeric - so we'll generate a numeric index for each distinct carrier string, and transform the dataframe to add that as a column

carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
numTrain = carrierIndexer.fit(train).transform(train)

# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "CarrierIdx", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(numTrain).select(col("features"), (col("ArrDelay").cast("Int").alias("label")))

training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,1.0,10.0,103...| 113|
[1.0,1.0,10.0,103...| 2|
[1.0,1.0,10.0,107...| -2|
[1.0,1.0,10.0,110...| 7|
[1.0,1.0,10.0,110...| -12|
[1.0,1.0,10.0,111...| 27|
[1.0,1.0,10.0,111...| 56|
[1.0,1.0,10.0,111...| 62|
[1.0,1.0,10.0,111...| 55|
[1.0,1.0,10.0,111...| 21|
[1.0,1.0,10.0,111...| -4|
[1.0,1.0,10.0,111...| 44|
[1.0,1.0,10.0,111...| -19|
[1.0,1.0,10.0,112...| 50|
[1.0,1.0,10.0,114...| -10|
[1.0,1.0,10.0,114...| 26|
[1.0,1.0,10.0,114...| 37|
[1.0,1.0,10.0,123...| 28|
[1.0,1.0,10.0,123...| -14|
[1.0,1.0,10.0,124...| -20|
+--------------------+-----+
only showing top 20 rows

### Train a Regression Model
Next, you need to train a regression model using the training data. To do this, create an instance of the regression algorithm you want to use and use its **fit** method to train a model based on the training DataFrame. In this exercise, you will use a *Linear Regression* algorithm - though you can use the same technique for any of the regression algorithms supported in the spark.ml API.

In [13]:
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)

model = lr.fit(training)

print ("Model trained!")

Model trained!

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **ArrDelay** column to **trueLabel**.

In [15]:
# Transform the test data to add the numeric carrier index
numTest = carrierIndexer.fit(test).transform(test)

# Generate the features vector and label
testing = assembler.transform(numTest).select(col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[1.0,1.0,10.0,105...| 41|
[1.0,1.0,10.0,107...| 30|
[1.0,1.0,10.0,107...| 30|
[1.0,1.0,10.0,110...| 82|
[1.0,1.0,10.0,111...| 86|
[1.0,1.0,10.0,111...| -10|
[1.0,1.0,10.0,111...| 54|
[1.0,1.0,10.0,124...| 0|
[1.0,1.0,10.0,124...| -28|
[1.0,1.0,10.0,132...| -8|
[1.0,1.0,10.0,134...| 91|
[1.0,1.0,10.0,143...| -20|
[1.0,1.0,10.0,153...| -24|
[1.0,1.0,2.0,1042...| 6|
[1.0,1.0,2.0,1069...| 26|
[1.0,1.0,2.0,1072...| 44|
[1.0,1.0,2.0,1127...| 36|
[1.0,1.0,2.0,1129...| -16|
[1.0,1.0,2.0,1129...| 12|
[1.0,1.0,2.0,1129...| 33|
+--------------------+---------+
only showing top 20 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict arrival delay for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted number of minutes late or early to the actual arrival delay.

In [17]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "trueLabel")

predicted.show()

+--------------------+--------------------+---------+
 features| prediction|trueLabel|
+--------------------+--------------------+---------+
[1.0,1.0,10.0,105...| 22.921129527821165| 41|
[1.0,1.0,10.0,107...| 28.410959072313613| 30|
[1.0,1.0,10.0,107...| 39.70286844580751| 30|
[1.0,1.0,10.0,110...| 95.15622570908282| 82|
[1.0,1.0,10.0,111...| 20.647478821230766| 86|
[1.0,1.0,10.0,111...| 1.6217019580062892| -10|
[1.0,1.0,10.0,111...| 60.664485973736134| 54|
[1.0,1.0,10.0,124...| 32.518490202356666| 0|
[1.0,1.0,10.0,124...| 4.039191259422566| -28|
[1.0,1.0,10.0,132...|-0.39872893001142096| -8|
[1.0,1.0,10.0,134...| 89.7283514572966| 91|
[1.0,1.0,10.0,143...| 6.51652918522758| -20|
[1.0,1.0,10.0,153...| -5.270017898193165| -24|
[1.0,1.0,2.0,1042...| 18.90976509016058| 6|
[1.0,1.0,2.0,1069...| 1.2709692244918784| 26|
[1.0,1.0,2.0,1072...| 40.61978716457527| 44|
[1.0,1.0,2.0,1127...| 1.9338364685682894| 36|
[1.0,1.0,2.0,1129...| 2.0010290405646938| -16|
[1.0,1.0,2.0,1129...| 24.86125432031921| 12|
[1.0,1.0,2.0,1129...| 34.09259681615674| 33|
+--------------------+--------------------+---------+
only showing top 20 rows

In [18]:
display(predicted)

features,prediction,trueLabel
"List(1, 6, List(), List(1.0, 1.0, 10.0, 10529.0, 14492.0, 20.0))",22.921129527821165,41
"List(1, 6, List(), List(1.0, 1.0, 10.0, 10721.0, 12478.0, 25.0))",28.410959072313613,30
"List(1, 6, List(), List(1.0, 1.0, 10.0, 10721.0, 12478.0, 37.0))",39.70286844580751,30
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11066.0, 12478.0, 96.0))",95.15622570908282,82
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 10529.0, 16.0))",20.647478821230766,86
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 13342.0, -3.0))",1.6217019580062892,-10
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 13930.0, 60.0))",60.664485973736134,54
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 10821.0, 29.0))",32.518490202356666,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 11433.0, -1.0))",4.039191259422566,-28
"List(1, 6, List(), List(1.0, 1.0, 10.0, 13244.0, 10423.0, -6.0))",-0.39872893001142096,-8


> Fin